In [21]:
import numpy as np
import pandas as pd
import os
import healpy as hp
from DS_data_transformation import get_prm
%config InlineBackend.figure_format = 'retina'
from DS_Planck_Unet import west_val, east_val, east
from matplotlib import pyplot as plt
from DS_Planck_Unet import load_planck_model, draw_pic_with_mask, draw_pic, val_pix
import os
from tqdm.notebook import tqdm
from DS_models_stat import stat_orig_cats, do_all_stats, stat_cat, cut_cat
from DS_detector import rematch_cat
from DS_healpix_fragmentation import cut_cat_by_pix

In [31]:
max_pred_unite = np.array([0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 ,
       0.65, 0.7 , 0.75, 0.775, 0.8 , 0.85, 0.9 , 0.905, 0.95])
max_pred_br = np.array([0,  5.05335105,  5.1069375 ,  5.1630591 ,  5.223298  ,  5.29437   ,
        5.3639328 ,  5.43414885,  5.5157134 ,  5.60974235,  5.711204  ,
        5.8320221 ,  5.9664762 ,  6.12062285,  6.2958075 ,  6.52273275,
        6.826352  ,  7.2506403 ,  8.0152959 , 10.0664315 , 10.5       ,
       12.        ])

In [23]:
def cut_max_pred(cat, max_pred):
    df = cat.copy()
    df = df[df['max_pred'] >= max_pred]
    df.index = np.arange(len(df))
    return df

In [24]:
pixels = {'41' : [41], '38' : [38], '6' : [6], 'e' : east_val, 'w' : west_val}

In [25]:
s = '/home/rt2122/Data/gen_cats/gen_all_found_brcat_prec0.2.csv'
def get_prec(s):
    return float(s[s.find('prec')+4:-4])
get_prec(s)

0.2

In [26]:
def get_cat_d(det_name, det_file, get_mpp_min=False, index_name='mpp'):
    cat_d = {}
    if det_name == 'SZcat(gen)':
        dirname = det_file 
        files = next(os.walk(dirname))[-1]
        cat_d = {get_prec(file) : pd.read_csv(os.path.join(dirname, file)) for file in files}
    else:
        det_cat = pd.read_csv(det_file)
        max_pred_vals = max_pred_unite
        if det_name in ['SZcat(y-map)', 'SZcat(y-map)_unique', 'inter_sn']:
            max_pred_vals = max_pred_br
        cat_d = {max_pred_prm : cut_max_pred(det_cat, max_pred_prm) for max_pred_prm in max_pred_vals}
        if get_mpp_min:
            new_cat_d = {}
            for val in cat_d:
                df = cat_d[val]
                new_cat_d[df[index_name].min()] = df
            cat_d = new_cat_d
    return cat_d

In [27]:
def get_recalls_file(det_name, det_file, big_pix, pixname, spec_precision=['PSZ2(z)', 'eROSITA'], index_name='max_pred',
                    dict_cut={}, get_mpp_min=False):
    cat_d = get_cat_d(det_name, det_file, get_mpp_min=get_mpp_min, index_name=index_name)
    recall_df = stat_cat(cat_d, big_pix=big_pix, read_det_files=False, match_dist=400/3600, 
                                     spec_precision=spec_precision, dict_cut=dict_cut)
    recall_df = recall_df.sort_index()
    recall_df.index.name = index_name
    filename = './check/{}_recall_{}_pix{}.csv'.format(det_name, index_name.replace('/', ''), pixname)
    recall_df.to_csv(filename)

In [28]:
cats_dict_max = {
            'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn.csv',
            #'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique.csv',
            #'inter' : '/home/rt2122/Data/inter_cats/inter.csv',
}
cats_dict_sn = {
            'SZcat(y-map)' : '/home/rt2122/Data/SRGz/brcat.csv',
            #'inter_sn' : '/home/rt2122/Data/inter_cats/inter_sn.csv',
            #'SZcat(y-map)_unique': '/home/rt2122/Data/inter_cats/brcat_unique.csv'
}
cats_dict_mpp = {
            'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn_mpp.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn_mpp.csv',
            'SZcat(y-map)' : '/home/rt2122/Data/SRGz/brcat_mpp.csv',
    
            'SZcat(gen)' : '/home/rt2122/Data/gen_cats/',
    
            #'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique_mpp.csv',
            #'inter' : '/home/rt2122/Data/inter_cats/inter_mpp.csv',
            #'SZcat(y-map)_unique': '/home/rt2122/Data/inter_cats/brcat_unique_mpp.csv'
}
cats_dict_mpp_er = {
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn_mpp_er.csv',
            'SZcat(y-map)' : '/home/rt2122/Data/SRGz/brcat_mpp_er.csv',
            'SZcat(gen)' : '/home/rt2122/Data/gen_cats_er/',
    
}

In [32]:
dict_by_index = zip([cats_dict_max, cats_dict_sn, cats_dict_mpp, cats_dict_mpp_er], ['max_pred', 's/n', 'mpp', 'mpp_er'])
for dict_cur, index_name in dict_by_index:
    if index_name != 's/n':
        continue
    for name in tqdm(dict_cur):
        print(name)
        get_mpp_min = False
        if name != 'SZcat(gen)' and (index_name == 'mpp' or index_name == 'mpp_er'):
            get_mpp_min = True
        get_recalls_file(name, dict_cur[name], list(range(48)), pixname='eb20',
                                   dict_cut={'b' : [20, np.inf], 'l' : [0, 180]}, index_name=index_name, get_mpp_min=get_mpp_min)
        for pixname in pixels:
            get_recalls_file(name, dict_cur[name], big_pix=pixels[pixname], pixname=pixname, dict_cut={}, index_name=index_name,
                            get_mpp_min=get_mpp_min)

SZcat(y-map)

